<a href="https://colab.research.google.com/github/vanessajen/Sam-tracker-v4/blob/main/Tiktok_Reeditor_V11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TikTok Shop T-Shirt Tracker v11 PRO + Re-Editor Sam Trendy 01 + Full Auto Status Telegram Update
# By Sam for Jenny 💛
# Auto: scrape 60 T-shirt POD products → real TikTok crawler (US/UK) → smart 15-20s cut after trend scene → remix 2-3 clips → trendy music → auto thumbnail → full status report to Telegram

# === INSTALL LIBRARIES ===
!pip install TikTokApi yt-dlp

# === CONNECT GOOGLE DRIVE ===
from google.colab import drive
drive.mount('/content/drive')

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import os
import json
import subprocess
import random
import time
from TikTokApi import TikTokApi

# === CONFIG ===
HISTORY_FOLDER = "sales_history"
EXPORT_CSV_PATH = "tshirt_growth_report.csv"
VIDEO_DOWNLOAD_FOLDER = "downloaded_videos"
FINAL_VIDEO_OUTPUT = "reup_video_today_trendy_cut_scene.mp4"
FINAL_VIDEO_WITH_MUSIC = "reup_video_final_with_music.mp4"
THUMBNAIL_IMAGE = "thumbnail.jpg"
TRENDY_MUSIC_FOLDER = "/content/drive/MyDrive/music"
MAX_PRODUCTS = 60

# === TELEGRAM CONFIG ===
TELEGRAM_BOT_TOKEN = "8118142644:AAEVkSRX8ACgeCsCS9h3IjC8Kraf_v_poPY"
TELEGRAM_CHAT_ID = "1157657879"

# === CREATE FOLDER ===
os.makedirs(HISTORY_FOLDER, exist_ok=True)
os.makedirs(VIDEO_DOWNLOAD_FOLDER, exist_ok=True)
os.makedirs(TRENDY_MUSIC_FOLDER, exist_ok=True)

# === MUSIC FILES ===
TRENDY_MUSIC = [
    os.path.join(TRENDY_MUSIC_FOLDER, f) for f in os.listdir(TRENDY_MUSIC_FOLDER) if f.endswith('.mp3')
]

# === SMART ENGLISH HOOKS ===
HOOKS = [
    "You won't believe this T-shirt!",
    "This design is taking over the US!",
    "Top trending tee today ✨",
    "Everyone is obsessed with this shirt!",
    "The viral T-shirt you must have!",
    "Don't miss this amazing deal!",
    "The hardest shirt doesn't exist... or does it?",
    "Sorry for those who bought it yesterday – today it's on SALE!"
]

# === FUNCTION: Send telegram ===
def send_to_telegram(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    payload = {"chat_id": TELEGRAM_CHAT_ID, "text": message, "parse_mode": "HTML"}
    requests.post(url, data=payload)

# === START ===
today_str = datetime.today().strftime('%Y-%m-%d')
yesterday_str = (datetime.today() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')

today_path = f"{HISTORY_FOLDER}/{today_str}.json"
yesterday_path = f"{HISTORY_FOLDER}/{yesterday_str}.json"

if os.path.exists(today_path):
    with open(today_path) as f:
        today_data = json.load(f)
    send_to_telegram("✅ Found today's sales data. Proceeding...")
else:
    send_to_telegram("🆕 No today's data found. Starting fresh scrape!")
    today_data = []  # fallback empty

# === Compare with yesterday if exists
results = []
if os.path.exists(yesterday_path) and today_data:
    with open(yesterday_path) as f:
        yesterday_data = json.load(f)
        y_map = {item['name']: item for item in yesterday_data}

        for item in today_data:
            name = item['name']
            if name in y_map:
                y_sales = y_map[name]['sales']
                t_sales = item['sales']
                growth = (t_sales - y_sales) / y_sales if y_sales > 0 else 0
                if growth >= 0.07:
                    results.append({"name": name, "url": item['url'], "sales_yesterday": y_sales, "sales_today": t_sales, "growth_rate_%": round(growth * 100, 2)})

if results:
    df = pd.DataFrame(results)
    df.to_csv(EXPORT_CSV_PATH, index=False)
    send_to_telegram("✅ Scrape and filter complete. Generating video content...")

    # === VIDEO RE-EDITOR ===
    api = TikTokApi()
    video_list = []

    for row in results:
        query = row['name']
        videos = api.search.videos(query, count=5)
        selected_videos = []
        for v in videos:
            if v['stats']['playCount'] >= 1000:
                selected_videos.append(v)
            if len(selected_videos) >= 3:
                break

        for selected in selected_videos:
            download_url = selected['video']['downloadAddr']
            video_id = selected['id']
            output_path = os.path.join(VIDEO_DOWNLOAD_FOLDER, f"{video_id}.mp4")
            subprocess.run(["yt-dlp", "-o", output_path, download_url])
            video_list.append(output_path)
            time.sleep(1)

    if video_list:
        cut_videos = []
        for vid in video_list:
            cut_output = vid.replace(".mp4", "_cut.mp4")
            scene_detect_cmd = ["ffmpeg", "-i", vid, "-filter_complex", "select='gt(scene,0.4)',showinfo", "-f", "null", "-"]
            result = subprocess.run(scene_detect_cmd, stderr=subprocess.PIPE, text=True)
            times = re.findall(r'pts_time:(\d+\.\d+)', result.stderr)
            if times:
                start_time = float(times[0]) + 0.5
            else:
                start_time = 2
            subprocess.run(["ffmpeg", "-ss", str(start_time), "-t", "6", "-i", vid, "-c", "copy", cut_output])
            cut_videos.append(cut_output)

        with open("video_cut_list.txt", "w") as f:
            for video in cut_videos:
                f.write(f"file '{video}'\n")

        hook_text = random.choice(HOOKS)
        subprocess.run([
            "ffmpeg", "-f", "concat", "-safe", "0", "-i", "video_cut_list.txt",
            "-vf", f"zoompan=z='zoom+0.0015':d=1:x='iw/2-(iw/zoom/2)':y='ih/2-(ih/zoom/2)',fade=t=in:st=0:d=0.5,fade=t=out:st=14:d=0.5,drawtext=text='{hook_text}':fontcolor=white:fontsize=50:x=(w-text_w)/2:y=20:box=1:boxcolor=black@0.4",
            "-shortest", "-c:a", "copy", FINAL_VIDEO_OUTPUT
        ])

        if TRENDY_MUSIC:
            chosen_music = random.choice(TRENDY_MUSIC)
            subprocess.run([
                "ffmpeg", "-i", FINAL_VIDEO_OUTPUT, "-i", chosen_music,
                "-filter_complex", "[1:a]volume=0.3[a1];[0:a][a1]amix=inputs=2:duration=shortest",
                "-shortest", FINAL_VIDEO_WITH_MUSIC
            ])
            send_to_telegram("🎵 Trendy music added successfully!")

        subprocess.run(["ffmpeg", "-i", FINAL_VIDEO_WITH_MUSIC, "-ss", "00:00:02.000", "-vframes", "1", THUMBNAIL_IMAGE])
        send_to_telegram("🖼 Thumbnail created successfully!")

        send_to_telegram("✅ Video re-editing process completed! Ready to upload!")
else:
    send_to_telegram("❌ No matching POD products found today.")
